In [198]:
import os
from pathlib import Path
from math import pi
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE


from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import (KFold, RandomizedSearchCV,
                                     cross_val_score, train_test_split)
from sklearn.tree import DecisionTreeClassifier

train_dir = Path('./data/train')
train_filepaths = list(train_dir.glob(r'**/*.jpg'))

test_dir = Path('./data/test')
test_filepaths = list(test_dir.glob(r'**/*.jpg'))

val_dir = Path('./data/validation')
val_filepaths = list(val_dir.glob(r'**/*.jpg'))

aug_dir = Path('./data/augmented/')
aug_filepaths = list(aug_dir.glob(r'**/*.jpg')) + list(aug_dir.glob(r'**/*.jpeg'))

data = train_filepaths + test_filepaths + val_filepaths

In [199]:
def paths_to_dataframe(path):
    labels = []
    for i in range(len(path)):
        labels.append(str(path[i]).split(os.sep)[-2])

    labels = pd.Series(labels, name='Label')
    path = pd.Series(path, name='Path').astype(str)

    df = pd.concat([path, labels], axis=1)

    df = df.sample(frac=0.5).reset_index(drop = True)

    return df
    
data_df = paths_to_dataframe(data)

In [200]:
data_df = data_df.sample(frac=1, random_state=42)
data_df.shape

(1725, 2)

In [201]:
k = 2
kf = KFold(n_splits=k, shuffle=True, random_state=42)

X = np.array(data_df['Path'])
y = np.array(data_df['Label'])

In [202]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

def metrics(y_test, y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    print("Accuracy: ", accuracy)
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F1: ", f1)

    classification_rep = classification_report(y_test, y_pred)
    print("Classification Report:")
    print(classification_rep)

    classes = np.unique(y_test)
    class_accuracy = {}
    for cls in classes:
        indices = np.where(y_test == cls)[0]
        class_accuracy[cls] = accuracy_score(y_test[indices], np.array(y_pred)[indices])

    file_path = './wynik.txt'
    with open(file_path, 'a') as file:
        file.write(str(accuracy))
        file.write('\n')
        file.write(str(precision))
        file.write('\n\n')


    return accuracy, precision, recall, f1, class_accuracy

In [203]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

def extract_features(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = model.predict(x)
    return features.flatten()

In [204]:
def sort_classes_by_gini_index(X_train, y_train):
    unique_labels = np.unique(y_train)
    gini_indices_per_class = {label: 0 for label in unique_labels}

    tree = DecisionTreeClassifier()
    tree.fit(X_train, y_train)

    feature_importances = tree.feature_importances_
    sorted_features = np.argsort(feature_importances)[::-1]  # Sort descending by feature importances

    for label in unique_labels:
        indices = np.where(y_train == label)[0]
        gini_indices_per_class[label] = np.mean(feature_importances[sorted_features[:len(indices)]]) if len(indices) > 0 else 0

    sorted_labels = sorted(unique_labels, key=lambda label: gini_indices_per_class[label], reverse=True)
    return sorted_labels

In [205]:
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from skopt import BayesSearchCV

def nested_dichotomy_without_removing(X_train, y_train, X_test):
    unique_labels = sort_classes_by_gini_index(X_train, y_train)
    classifiers = []

    for i in unique_labels:
        y_binary = np.where(y_train == i, 1, 0)
        tree = DecisionTreeClassifier()
        tree.fit(X_train, y_binary)
        classifiers.append((tree, i))

        print(i)

    predictions = []
    for tree, positive_label in classifiers:
        binary_prediction = tree.predict(X_test)
        predictions.append(np.where(binary_prediction == 1, positive_label, None))

    results = [None for _ in range(len(X_test))]
    for i in range(len(predictions)):
        for j in range(len(X_test)):
            if predictions[i][j] is not None and results[j] == None:
                results[j] = unique_labels[i]

    for i in range(len(results)):
        if results[i] is None:
            results[i] = unique_labels[0]
        

    return results, predictions, classifiers

In [206]:
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from skopt import BayesSearchCV

def nested_dichotomy_with_removing(X_train, y_train, X_test):
    unique_labels = sort_classes_by_gini_index(X_train, y_train)
    classifiers = []

    for i in unique_labels:
        y_binary = np.where(y_train == i, 1, 0)
        tree = DecisionTreeClassifier()
        tree.fit(X_train, y_binary)
        classifiers.append((tree, i))

        indices_to_remove = np.where(y_train == i)[0]
        X_train = np.delete(X_train, indices_to_remove, axis=0)
        y_train = np.delete(y_train, indices_to_remove)

        print(i, '   removing')
        print(X_train.shape)

    predictions = []
    for tree, positive_label in classifiers:
        binary_prediction = tree.predict(X_test)
        predictions.append(np.where(binary_prediction == 1, positive_label, None))

    results = [None for _ in range(len(X_test))]
    for i in range(len(predictions)):
        for j in range(len(X_test)):
            if predictions[i][j] is not None and results[j] == None:
                results[j] = unique_labels[i]

    for i in range(len(results)):
        if results[i] is None:
            results[i] = unique_labels[0]

    return results, predictions, classifiers

In [207]:
from tensorflow.keras.applications import VGG16
vgg16_model = VGG16(weights='imagenet', include_top=False, pooling='avg')
X_ext = np.array([extract_features(img_path, vgg16_model) for img_path in X])

/var/folders/kb/21stffp561n7vlhn9h_r55zc0000gn/T/ipykernel_47887/1684988159.py:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  mobilenet_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 468ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━

In [208]:
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks, RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTE
from tensorflow.keras.applications import MobileNetV2


def experiment_loop():

    scores = [[] for _ in range(k)]
    scores_rem = [[] for _ in range(k)]

    # mobilenet_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')
    # X_ext = np.array([extract_features(img_path, mobilenet_model) for img_path in X])

    for fold, (train_index, val_index) in enumerate(kf.split(X)):
        X_train, X_val = X_ext[train_index], X_ext[val_index]
        y_train, y_val = y[train_index], y[val_index]

        results, pred, models = nested_dichotomy_without_removing(X_train, y_train, X_val)
        results_rem, pred_rem, models_rem = nested_dichotomy_with_removing(X_train, y_train, X_val)
        accuracy, precision, recall, f1, class_accuracy = metrics(y_val, results)
        accuracy_rem, precision_rem, recall_rem, f1_rem, class_accuracy = metrics(y_val, results_rem)
        scores[fold].append((accuracy, precision, recall, f1, class_accuracy))
        scores_rem[fold].append((accuracy_rem, precision_rem, recall_rem, f1_rem, class_accuracy))

    return scores, scores_rem

scores, scores_rem = experiment_loop()

eggplant
banana
potato
apple
garlic
lemon
watermelon
cabbage
kiwi
ginger
grapes
lettuce
orange
soy beans
tomato
capsicum
corn
mango
pomegranate
raddish
chilli pepper
peas
pineapple
jalepeno
spinach
bell pepper
carrot
paprika
cauliflower
pear
turnip
onion
sweetpotato
cucumber
sweetcorn
beetroot
eggplant    removing
(847, 1280)
banana    removing
(831, 1280)
potato    removing
(813, 1280)
apple    removing
(794, 1280)
garlic    removing
(775, 1280)
lemon    removing
(755, 1280)
watermelon    removing
(735, 1280)
cabbage    removing
(714, 1280)
kiwi    removing
(693, 1280)
ginger    removing
(671, 1280)
grapes    removing
(649, 1280)
lettuce    removing
(627, 1280)
orange    removing
(605, 1280)
soy beans    removing
(583, 1280)
tomato    removing
(561, 1280)
capsicum    removing
(538, 1280)
corn    removing
(515, 1280)
mango    removing
(492, 1280)
pomegranate    removing
(469, 1280)
raddish    removing
(446, 1280)
chilli pepper    removing
(422, 1280)
peas    removing
(398, 1280)
pineap

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from itertools import combinations
from scipy.stats import ttest_rel

param_methods = ['Nested Dichotomy Without Removing', 'Nested Dichotomy With Removing']

class_accuracies = [[fold[0][-1] for fold in scores], [fold[0][-1] for fold in scores_rem]]
sum_class_accuracy = [{key: 0 for key in class_accuracies[model_idx][0].keys()} for model_idx in range(len(class_accuracies))]


for model_idx, model_accuracies in enumerate(class_accuracies):
    for fold in model_accuracies:
        for key, value in fold.items():
            if key in sum_class_accuracy[model_idx]:
                sum_class_accuracy[model_idx][key] += value


num_folds = len(scores)


avg_class_accuracy = [{key: value / num_folds for key, value in sum_accuracies.items()} for sum_accuracies in sum_class_accuracy]


for model_idx, method_name in enumerate(param_methods):
    print("Average Class Accuracy for", method_name, ":", avg_class_accuracy[model_idx])

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.bar(avg_class_accuracy[0].keys(), avg_class_accuracy[0].values(), color='skyblue')
plt.title('Accuracy for Each Class (' + param_methods[0] + ')')
plt.xlabel('Classes')
plt.ylabel('Accuracy')
plt.xticks(rotation=90, ha='right')
plt.ylim(0, 1)

plt.subplot(1, 2, 2)
plt.bar(avg_class_accuracy[1].keys(), avg_class_accuracy[1].values(), color='lightgreen')
plt.title('Accuracy for Each Class (' + param_methods[1] + ')')
plt.xlabel('Classes')
plt.ylabel('Accuracy')
plt.xticks(rotation=90, ha='right')
plt.ylim(0, 1)

plt.tight_layout()
plt.show()


In [ ]:
avg_accuracy = []
avg_precision = []
avg_recall = []
avg_f1 = []


for idx, method_name in enumerate(param_methods):

    method_scores = scores if idx == 0 else scores_rem

    accuracy = [score[0][0] for score in method_scores]
    precisions = [score[0][1] for score in method_scores]
    recalls = [score[0][2] for score in method_scores]
    f1_scores = [score[0][3] for score in method_scores]

    avg_accuracy.append(np.mean(accuracy))
    avg_precision.append(np.mean(precisions))
    avg_recall.append(np.mean(recalls))
    avg_f1.append(np.mean(f1_scores))


for idx, method_name in enumerate(param_methods):
    print("Average Accuracy for", method_name + ":", avg_accuracy[idx])
    print("Average Precision for", method_name + ":", avg_precision[idx])
    print("Average Recall for", method_name + ":", avg_recall[idx])
    print("Average F1-Score for", method_name + ":", avg_f1[idx])


plt.figure(figsize=(14, 10))
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
values = [avg_accuracy, avg_precision, avg_recall, avg_f1]

for i in range(len(metrics)):
    plt.subplot(2, 2, i+1)
    plt.bar(param_methods, values[i], color=['skyblue', 'lightgreen'])
    plt.title('Average ' + metrics[i])
    plt.xlabel('Feature extraction method')
    plt.ylabel(metrics[i])
    plt.ylim(0, 1)
    plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()


In [ ]:
def compare_methods(method1, name1, method2, name2):
    t_statistic, p_value = ttest_rel(method1, method2)
    comparison = f"{name1} is better than {name2}" if t_statistic > 0 else f"{name2} is better than {name1}"
    significance = "significant" if p_value < 0.05 else "not significant"
    return t_statistic, p_value, comparison, significance


methods = param_methods
values = [avg_accuracy, avg_precision, avg_recall, avg_f1]

pairs = combinations(enumerate(methods), 2)


for pair in pairs:
    (idx_1, name_1), (idx_2, name_2) = pair
    t_statistic, p_value, comparison, significance = compare_methods(values[idx_1], name_1, values[idx_2], name_2)
    print(f'{name_1} vs. {name_2}:')
    print(f'T-statistic: {t_statistic}')
    print(f'P-value: {p_value}')
    print(f'Result: {comparison} (p-value {significance})')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_radar_chart(methods, avg_accuracy, avg_precision, avg_recall, avg_f1, metrics):
    num_features = 4
    angles = np.linspace(0, 2 * np.pi, num_features, endpoint=False).tolist()
    angles += angles[:1]

    fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))

    for method, avg_acc, avg_prec, avg_rec, avg_f in zip(methods, avg_accuracy, avg_precision, avg_recall, avg_f1):
        values = [avg_acc, avg_prec, avg_rec, avg_f, avg_acc]
        ax.plot(angles, values, linewidth=1, linestyle='solid', label=method)
        ax.fill(angles, values, alpha=0.25)

    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(metrics)
    ax.set_yticklabels([])

    ax.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))

    plt.show()

metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
plot_radar_chart(param_methods, avg_accuracy, avg_precision, avg_recall, avg_f1, metrics)
